In [1]:
import re
import time
import pandas as pd
from eunjeon import Mecab
from numpy import dot
from numpy.linalg import norm
import numpy as np
import pickle
import gzip
from collections import Counter

## **define**

In [2]:
def processing(content):
    result = re.sub(r'[a-zA-Z가-힣]+뉴스', '', str(content))
    result = re.sub(r'[a-zA-Z가-힣]+ 뉴스', '', result)
    result = re.sub(r'[a-zA-Z가-힣]+newskr', '', result)
    result = re.sub(r'[a-zA-Z가-힣]+Copyrights', '', result)    
    result = re.sub(r'[a-zA-Z가-힣]+ Copyrights', '', result)
    result = re.sub(r'\s+Copyrights', '', result)
    result = re.sub(r'[a-zA-Z가-힣]+com', '', result)
    result = re.sub(r'[가-힣]+ 기자', '', result)    
    result = re.sub(r'[가-힣]+기자', '', result)
    result = re.sub(r'[가-힣]+ 신문', '', result)    
    result = re.sub(r'[가-힣]+신문', '', result)
    result = re.sub(r'데일리+[가-힣]', '', result)
    result = re.sub(r'[가-힣]+투데이', '', result)    
    result = re.sub(r'[가-힣]+미디어', '', result)
    result = re.sub(r'[가-힣]+ 데일리', '', result)
    result = re.sub(r'[가-힣]+데일리', '', result)
    result = re.sub(r'[가-힣]+ 콘텐츠 무단', '', result)
    result = re.sub(r'전재\s+변형', '전재', result)
    result = re.sub(r'[가-힣]+ 전재', '', result)    
    result = re.sub(r'[가-힣]+전재', '', result)
    result = re.sub(r'[가-힣]+배포금지', '', result)
    result = re.sub(r'[가-힣]+배포 금지', '', result)
    result = re.sub(r'\s+배포금지', '', result)
    result = re.sub(r'\s+배포 금지', '', result)    
    result = re.sub(r'[a-zA-Z가-힣]+.kr', '', result)
    result = re.sub(r'/^[a-z0-9_+.-]+@([a-z0-9-]+\.)+[a-z0-9]{2,4}$/', '', result)
    result = re.sub(r'[\r|\n]', '', result)
    result = re.sub(r'\[[^)]*\]', '', result)
    result = re.sub(r'\([^)]*\)', '', result)
    result = re.sub(r'[^ ㄱ-ㅣ가-힣A-Za-z0-9]', '', result)
    result = result.replace('뉴스코리아','').replace('및','').replace('Copyright', '').replace('저작권자','').replace('ZDNET A RED VENTURES COMPANY','')    
    result = result.strip()

    return result

In [3]:
def tokenize(dataframe):
    start = time.time()
    m = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
    nouns = []

    for i in range(len(dataframe)):
        temp_noun = m.nouns(str(dataframe['content'][i]))
        nouns.append(temp_noun)

    print("소요 시간 :", time.time() - start,"초") 
    
    return nouns

## **data processing**

In [4]:
predata = pd.read_csv('/workspace/data/git_category_initial.csv')
predata

,Unnamed: 0,category,content,sp3,sp2,sp1
0,0,"[11, 3]",열과 전기가 잘 통해 산업계의 금으로 불리는 희귀 광물인 희토류영구자석 형태로 전기...,NaN,에너지,과학
1,1,"[9, 23, 24]",최근 열린 미국의회 청문회에서 몇몇 앱개발사가 애플과 구글은 플랫폼 영향력 측면에서...,광고,경제,개발
2,2,[29],서효원씨 별세 서동면 서동윤 서동신씨 부친상 김은미씨 시부상 김성덕씨 장인상...,NaN,NaN,etc
3,3,"[23, 13, 5]",비트코인 등 암호화폐 시장이 급락하면서 투자자들이 패닉 상태에 빠졌다 국내 주...,투자,블록체인,경제
4,4,"[24, 17, 23]",웰컴저축은행은 꿈테크 프로젝트 런포드림 영상이 세계 대 국제 광고제인 클리오 어워즈...,경제,미디어,광고
...,...,...,...,...,...,...
1173,1197,[10],퀄컴이 분기 실적 호조를 이어갔다 코로나가 종식되지 않았지만 스마트폰 시장은 예년...,NaN,NaN,통신
1174,1198,[10],아파트 각 가정에 인터넷을 연결해주는 통신 장비입니다 알파벳 U 표시는 이 장비가...,NaN,NaN,통신
1175,1199,[26],이번 주에도 유통식음료 업계는 큰 사건이 없었습니다 지난주 남양유업 사태와 같은 빅...,NaN,NaN,O2O
1176,1201,[26],콘텐츠 커뮤니케이션 기업 유에이와 뉴미디어 컨텐츠 제작기업 비엠미디어가 전통시장의 ...,NaN,NaN,O2O


In [5]:
predata = predata[['category','content']]
predata

,category,content
0,"[11, 3]",열과 전기가 잘 통해 산업계의 금으로 불리는 희귀 광물인 희토류영구자석 형태로 전기...
1,"[9, 23, 24]",최근 열린 미국의회 청문회에서 몇몇 앱개발사가 애플과 구글은 플랫폼 영향력 측면에서...
2,[29],서효원씨 별세 서동면 서동윤 서동신씨 부친상 김은미씨 시부상 김성덕씨 장인상...
3,"[23, 13, 5]",비트코인 등 암호화폐 시장이 급락하면서 투자자들이 패닉 상태에 빠졌다 국내 주...
4,"[24, 17, 23]",웰컴저축은행은 꿈테크 프로젝트 런포드림 영상이 세계 대 국제 광고제인 클리오 어워즈...
...,...,...
1173,[10],퀄컴이 분기 실적 호조를 이어갔다 코로나가 종식되지 않았지만 스마트폰 시장은 예년...
1174,[10],아파트 각 가정에 인터넷을 연결해주는 통신 장비입니다 알파벳 U 표시는 이 장비가...
1175,[26],이번 주에도 유통식음료 업계는 큰 사건이 없었습니다 지난주 남양유업 사태와 같은 빅...
1176,[26],콘텐츠 커뮤니케이션 기업 유에이와 뉴미디어 컨텐츠 제작기업 비엠미디어가 전통시장의 ...


In [10]:
predata['content'] = predata.content.apply(processing)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
predata['content'][219]

'LG전자가 내달 년형 LG 그램 를 출시한다고 일 밝혔다 회사는 기존   형에 이어 형대 제품을 추가로 선보이며 화면 크기별로 촘촘한 라인업을 갖추게 된다LG전자는 신제품에 올해 앞서 선보인  화면비의 제품들과는 달리  화면비의 형 FHD IPS 디스플레이를 탑재했다 기존  화면 비에 익숙한 고객들 요구도 만족시켰다 여기에 초슬림 베젤과 퍼펙트 히든 힌지를 적용 기존 그램 형과 비교해 콤팩트한 디자인을 구현했다LG 그램 는 그램의 가벼운 무게에 와트시 대용량 배터리를 탑재했다 최신 인텔 세대 프로세서 타이거레이크를 적용해 기존 세대 프로세서 대비 데이터 처리 속도가 약  빨라졌다인텔 내장 그래픽 프로세서 아이리스 엑스이도 장착해 고화질 영상 작업과 게임 구동 시 빠른 속도로 쾌적한 환경을 제공한다LG전자는 올해 새롭게 선보인 투인원 노트북 그램  형 제품에 옵시디안 블랙 형에 쿼츠 실버 색상을 추가하며 색상 라인업도 늘린다 지난 월 그램  출시 당시 형은 토파즈 그린 형은 옵시디안 블랙 색상을 출시한 바 있다그램 은 고객의 상황에 따라 노트북 태블릿 모드 등으로 바꿔 사용할 수 있는 제품이다 예를 들어 고객은 문서 작업이나 영상 편집 등 키보드를 사용할 때에는 노트북 모드로 사용하고 영상 콘텐츠를 즐기거나 펜을 활용할 때는 태블릿 모드로 사용할 수 있어 생산성이 극대화된다LG전자는 월 한 달간 더 큰 그램이 더 크게 드리는 감사 이벤트를 실시한다 이 기간 구매 고객에는 노트북 파우치 게이밍 헤드셋 커블체어 등을 추가로 증정한다김선형 LG전자 한국영업본부 HE마케팅담당은 라인업 확대로 고객은 본인의 사용환경이나 색상 등 디자인 선호도에 따라 LG 그램을 구매할 수 있게 됐다며 고객에게 다양한 선택지를 제공해 코로나 로 인해 성장하고 있는 노트북 시장에서 주도권을 더욱 강화할 것이라고'

In [12]:
predata['content_token'] = tokenize(predata)
predata

소요 시간 : 2.3212263584136963 초


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,category,content,content_token
0,"[11, 3]",열과 전기가 잘 통해 산업계의 금으로 불리는 희귀 광물인 희토류영구자석 형태로 전기...,"[열, 전기, 산업, 금, 광물, 희토류, 영구, 자석, 형태, 전기차, 모터, 풍..."
1,"[9, 23, 24]",최근 열린 미국의회 청문회에서 몇몇 앱개발사가 애플과 구글은 플랫폼 영향력 측면에서...,"[최근, 미국, 의회, 청문회, 앱, 개발, 사, 애플, 구글, 플랫, 폼, 영향력..."
2,[29],서효원씨 별세 서동면 서동윤 서동신씨 부친상 김은미씨 시부상 김성덕씨 장인상 일 삼...,"[서효원, 씨, 별세, 서동면, 서동, 윤, 서동신, 씨, 부친상, 김은미, 씨, ..."
3,"[23, 13, 5]",비트코인 등 암호화폐 시장이 급락하면서 투자자들이 패닉 상태에 빠졌다 국내 주요 암...,"[비트코인, 등, 암호, 화폐, 시장, 급락, 투자자, 패닉, 상태, 국내, 주요,..."
4,"[24, 17, 23]",웰컴저축은행은 꿈테크 프로젝트 런포드림 영상이 세계 대 국제 광고제인 클리오 어워즈...,"[웰컴, 저축, 은행, 꿈, 테크, 프로젝트, 런, 포드, 림, 영상, 세계, 국제..."
...,...,...,...
1173,[10],퀄컴이 분기 실적 호조를 이어갔다 코로나가 종식되지 않았지만 스마트폰 시장은 예년 ...,"[퀄컴, 분기, 실적, 호조, 코로나, 종식, 스마트폰, 시장, 예년, 수준, 일,..."
1174,[10],아파트 각 가정에 인터넷을 연결해주는 통신 장비입니다 알파벳 U 표시는 이 장비가 ...,"[아파트, 가정, 인터넷, 연결, 통신, 장비, 알파벳, 표시, 장비, 지원, 속도..."
1175,[26],이번 주에도 유통식음료 업계는 큰 사건이 없었습니다 지난주 남양유업 사태와 같은 빅...,"[이번, 주, 유통, 식음료, 업계, 사건, 지난주, 남양유업, 사태, 빅, 이벤트..."
1176,[26],콘텐츠 커뮤니케이션 기업 유에이와 컨텐츠 제작기업 가 전통시장의 디지털 전환을 선...,"[콘텐츠, 커뮤니케이션, 기업, 유, 에이, 컨텐츠, 제작, 기업, 전통, 시장, ..."


In [13]:
# data split
predata['sp1'] = predata.category.str.split(',').str[0]
predata['sp2'] = predata.category.str.split(',').str[1]
predata['sp3'] = predata.category.str.split(',').str[2]

predata['sp1'] = predata.sp1.str.replace('[','')
predata['sp1'] = predata.sp1.str.replace(']','')
predata['sp2'] = predata.sp2.str.replace('[','')
predata['sp2'] = predata.sp2.str.replace(']','')
predata['sp3'] = predata.sp3.str.replace('[','')
predata['sp3'] = predata.sp3.str.replace(']','')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
predata['content_token'][1]

['최근',
 '미국',
 '의회',
 '청문회',
 '앱',
 '개발',
 '사',
 '애플',
 '구글',
 '플랫',
 '폼',
 '영향력',
 '측면',
 '존재',
 '우려',
 '글',
 '애플',
 '세계',
 '모바일',
 '운영',
 '체제',
 '안드로이드',
 '개발',
 '플랫',
 '폼',
 '업체',
 '마이크',
 '소프트',
 '아마존',
 '페이스북',
 '등',
 '온라인',
 '플랫',
 '폼',
 '시장',
 '이익',
 '독식',
 '지적',
 '각국',
 '규제',
 '기관',
 '규제',
 '강화',
 '거대',
 '기업',
 '겨냥',
 '반독점',
 '법',
 '위반',
 '여부',
 '조사',
 '시작',
 '애플',
 '구글',
 '앱',
 '경제',
 '측면',
 '불공정',
 '거래',
 '행위',
 '조사',
 '모바일',
 '앱',
 '장터',
 '시장',
 '규모',
 '조',
 '달러',
 '애플',
 '앱',
 '스토어',
 '수수료',
 '수천억',
 '달러',
 '수입',
 '년',
 '애플',
 '콘텐츠',
 '서비스',
 '부문',
 '연간',
 '천억',
 '달러',
 '최근',
 '황금',
 '거위',
 '앱',
 '스토어',
 '회사',
 '골치',
 '거리',
 '앱',
 '개발자',
 '소비자',
 '규제',
 '기관',
 '애플',
 '앱',
 '스토어',
 '구글',
 '플레이',
 '스토어',
 '수수료',
 '정책',
 '문제',
 '제기',
 '유명',
 '인기',
 '게임',
 '포티',
 '나이트',
 '제작사',
 '에픽',
 '게임즈',
 '최근',
 '애플',
 '앱',
 '스토어',
 '구글',
 '플레이',
 '스토어',
 '수수료',
 '애플',
 '구글',
 '제소',
 '소비자',
 '앱',
 '스토어',
 '자의',
 '고강',
 '통제',
 '자유',
 '시장',
 '경쟁',
 '저해',
 '불만',
 '유럽연합',
 '집행',
 '위원회',
 '애플',

In [16]:
# pickle save
with open('/workspace/data/processing/git_category_predata.pickle', 'wb') as f:
    pickle.dump(predata, f, pickle.HIGHEST_PROTOCOL)

## **jaccard similarity**

In [6]:
data = pd.read_csv('/workspace/data/git_category.csv')
data

,Unnamed: 0,category,content
0,0,"[23, 20, 9]",1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품...
1,1,"[0, 9, 21]",서창호 KAIST 전기전자공학부 교수가 국제전기전자공학회 정보이론학회에서 수여하는 ...
2,2,[5],손정의 일본 소프트뱅크 회장의 비전펀드가 중국발 악재에 막대한 손실을 떠안게 됐다 ...
3,3,[5],장병규 의장 IPO 기자간담회크래프톤 IPO 온라인 간담회에서 장병규 크래프톤 의장...
4,4,"[3, 9]",LG디스플레이가 ESG 경영 강화 목적으로 한국전력의 녹색프리미엄 제도를 활용해 태...
...,...,...,...
251508,499995,[21],아버지를 닮은 아버지 같은 해군 간부가 되겠다우석대 진천캠퍼스 군사안보학과 김해나 ...
251509,499996,[9],평택시 기록관은 시민의 생활상과 추억이 담긴 민간기록물을 오는 10월 29일까지 수...
251510,499997,[9],국립낙동강생물자원관 연구진경북 상주시에 있는 환경부 산하 국립낙동강생물자원관이 최근...
251511,499998,[29],총 상금 1200만 원 규모 8개 과채 소재 사진과 동영상 카드뉴스로 응모전북농협전...


In [7]:
data['content']=data.content.apply(processing)
data

KeyboardInterrupt: 

In [ ]:
data['content_token'] = tokenize(data)
data

In [15]:
# pickle save
with open('/workspace/data/processing/git_category.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

# save and compress.
# with gzip.open('/workspace/data/processing/git_category.pickle', 'wb') as f:
#     pickle.dump(data, f)

# load and uncompress.
# with gzip.open('testPickleFile.pickle','rb') as f:
#     data = pickle.load(f)    

## **dataload start**

In [4]:
def most_count(predata):    
    for i in range(30):
        globals()['c{}'.format(i)] = predata[(predata.category.apply(lambda x: i in eval(x)))]
        globals()['c{}'.format(i)] = sum(globals()['c{}'.format(i)]['content_token'].to_list(), [])
        globals()['c{}'.format(i)] = Counter(globals()['c{}'.format(i)])
        globals()['c{}'.format(i)] = dict(globals()['c{}'.format(i)].most_common(200)).keys()
        globals()['c{}'.format(i)] = list(globals()['c{}'.format(i)])

In [5]:
def jaccard_similarity(data, count):
    union = set(data).union(set(count))
    intersection = set(data).intersection(set(count))
    result = len(intersection)/len(union)
    
    return result

In [6]:
def remove_common():
    try:
        common=[]
        for i in range(30):
            common.append(globals()['c{}'.format(i)])
        result = set(sum(common,[]))


        for a in result:    
            for i in range(30):
                globals()['c{}'.format(i)].remove(a)
                
    except ValueError:
        pass

In [30]:
def labeling(token):
    result={}    
    for i in range(30):
        result[i] = jaccard_similarity(token, globals()['c{}'.format(i)])

    result = sorted(result.items(), key=lambda x:x[1], reverse=True)[:3]
    result = dict(result)
    result = list(result.keys())

    return result

In [43]:
# load
with open('/workspace/data/processing/git_category.pickle', 'rb') as f:
    data = pickle.load(f)

with open('/workspace/data/processing/git_category_predata.pickle', 'rb') as f:
    predata = pickle.load(f)

In [44]:
predata

,category,content,content_token,sp1,sp2,sp3
0,"[11, 3]",열과 전기가 잘 통해 산업계의 금으로 불리는 희귀 광물인 희토류영구자석 형태로 전기...,"[열, 전기, 산업, 금, 광물, 희토류, 영구, 자석, 형태, 전기차, 모터, 풍...",11,3,NaN
1,"[9, 23, 24]",최근 열린 미국의회 청문회에서 몇몇 앱개발사가 애플과 구글은 플랫폼 영향력 측면에서...,"[최근, 미국, 의회, 청문회, 앱, 개발, 사, 애플, 구글, 플랫, 폼, 영향력...",9,23,24
2,[29],서효원씨 별세 서동면 서동윤 서동신씨 부친상 김은미씨 시부상 김성덕씨 장인상 일 삼...,"[서효원, 씨, 별세, 서동면, 서동, 윤, 서동신, 씨, 부친상, 김은미, 씨, ...",29,NaN,NaN
3,"[23, 13, 5]",비트코인 등 암호화폐 시장이 급락하면서 투자자들이 패닉 상태에 빠졌다 국내 주요 암...,"[비트코인, 등, 암호, 화폐, 시장, 급락, 투자자, 패닉, 상태, 국내, 주요,...",23,13,5
4,"[24, 17, 23]",웰컴저축은행은 꿈테크 프로젝트 런포드림 영상이 세계 대 국제 광고제인 클리오 어워즈...,"[웰컴, 저축, 은행, 꿈, 테크, 프로젝트, 런, 포드, 림, 영상, 세계, 국제...",24,17,23
...,...,...,...,...,...,...
1173,[10],퀄컴이 분기 실적 호조를 이어갔다 코로나가 종식되지 않았지만 스마트폰 시장은 예년 ...,"[퀄컴, 분기, 실적, 호조, 코로나, 종식, 스마트폰, 시장, 예년, 수준, 일,...",10,NaN,NaN
1174,[10],아파트 각 가정에 인터넷을 연결해주는 통신 장비입니다 알파벳 U 표시는 이 장비가 ...,"[아파트, 가정, 인터넷, 연결, 통신, 장비, 알파벳, 표시, 장비, 지원, 속도...",10,NaN,NaN
1175,[26],이번 주에도 유통식음료 업계는 큰 사건이 없었습니다 지난주 남양유업 사태와 같은 빅...,"[이번, 주, 유통, 식음료, 업계, 사건, 지난주, 남양유업, 사태, 빅, 이벤트...",26,NaN,NaN
1176,[26],콘텐츠 커뮤니케이션 기업 유에이와 컨텐츠 제작기업 가 전통시장의 디지털 전환을 선...,"[콘텐츠, 커뮤니케이션, 기업, 유, 에이, 컨텐츠, 제작, 기업, 전통, 시장, ...",26,NaN,NaN


In [45]:
data

,Unnamed: 0,category,content,content_token
0,0,"[23, 20, 9]",1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품...,"[강소, 기업, 기술, 독립, 전쟁, 에버, 켐, 텍, 년, 친환경, 식품, 포장재..."
1,1,"[0, 9, 21]",서창호 KAIST 전기전자공학부 교수가 국제전기전자공학회 정보이론학회에서 수여하는 ...,"[서창호, 전기, 전자, 공학부, 교수, 국제, 전기, 전자, 공학, 정보, 이론,..."
2,2,[5],손정의 일본 소프트뱅크 회장의 비전펀드가 중국발 악재에 막대한 손실을 떠안게 됐다 ...,"[손정, 일본, 소프트, 뱅크, 회장, 비전, 펀드, 중국, 발, 악재, 손실, 펀..."
3,3,[5],장병규 의장 IPO 기자간담회크래프톤 IPO 온라인 간담회에서 장병규 크래프톤 의장...,"[장병규, 의장, 기자, 간담, 회크, 래프, 톤, 온라인, 간담회, 장병규, 크,..."
4,4,"[3, 9]",LG디스플레이가 ESG 경영 강화 목적으로 한국전력의 녹색프리미엄 제도를 활용해 태...,"[디스플레이, 경영, 강화, 목적, 한국전력, 녹색, 프리미엄, 제도, 활용, 태양..."
...,...,...,...,...
251508,499995,[21],아버지를 닮은 아버지 같은 해군 간부가 되겠다우석대 진천캠퍼스 군사안보학과 김해나 ...,"[아버지, 아버지, 해군, 간부, 석대, 진천, 캠퍼스, 군사, 보학, 김해, 학생..."
251509,499996,[9],평택시 기록관은 시민의 생활상과 추억이 담긴 민간기록물을 오는 10월 29일까지 수...,"[평택시, 기록관, 시민, 생활상, 추억, 민간, 기록물, 월, 일, 수집, 일, ..."
251510,499997,[9],국립낙동강생물자원관 연구진경북 상주시에 있는 환경부 산하 국립낙동강생물자원관이 최근...,"[국립, 낙동강, 생물, 자원, 관, 연구, 진경, 북, 상주시, 환경부, 산하, ..."
251511,499998,[29],총 상금 1200만 원 규모 8개 과채 소재 사진과 동영상 로 응모전북농협전북농협은...,"[상금, 만, 원, 규모, 개, 과채, 소재, 사진, 동영상, 응모, 전북, 농협,..."


In [46]:
# 약 1100 레이블 데이터를 카테고리 별 토큰 set을 생성해서 count로 상위 10개 or count 10개 이상(예)
# category_1 = predata[(predata.category.apply(lambda x: 1 in eval(x)))]
# c1 = sum(category_1['content_token'].to_list(), [])
# c1
# eval(predata.iloc[0, 0])

# 카테고리 별 상위 100개 토큰
# c1 = Counter(c1)
# c1 = c1.most_common(100)
# c1 = dict(c1)
#c1 = list(c1.keys())

In [47]:
# global variable
most_count(predata)

In [48]:
remove_common()

In [49]:
# token = data.content_token.values.tolist()
# result={}
# for i in range(30):
#     result[i] = jaccard_similarity(token[5], globals()['c{}'.format(i)])
# result = sorted(result.items(), key=lambda x:x[1], reverse=True)[:3]
# result = dict(result)
# print(result)

In [50]:
data['label'] = data.content_token.apply(labeling)

In [53]:
data.rename(columns = {'category': 'category_old', 'label': 'category'}, inplace = True)

In [54]:
data

,Unnamed: 0,category_old,content,content_token,category
0,0,"[23, 20, 9]",1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품...,"[강소, 기업, 기술, 독립, 전쟁, 에버, 켐, 텍, 년, 친환경, 식품, 포장재...","[2, 3, 9]"
1,1,"[0, 9, 21]",서창호 KAIST 전기전자공학부 교수가 국제전기전자공학회 정보이론학회에서 수여하는 ...,"[서창호, 전기, 전자, 공학부, 교수, 국제, 전기, 전자, 공학, 정보, 이론,...","[3, 21, 22]"
2,2,[5],손정의 일본 소프트뱅크 회장의 비전펀드가 중국발 악재에 막대한 손실을 떠안게 됐다 ...,"[손정, 일본, 소프트, 뱅크, 회장, 비전, 펀드, 중국, 발, 악재, 손실, 펀...","[5, 23, 13]"
3,3,[5],장병규 의장 IPO 기자간담회크래프톤 IPO 온라인 간담회에서 장병규 크래프톤 의장...,"[장병규, 의장, 기자, 간담, 회크, 래프, 톤, 온라인, 간담회, 장병규, 크,...","[23, 5, 29]"
4,4,"[3, 9]",LG디스플레이가 ESG 경영 강화 목적으로 한국전력의 녹색프리미엄 제도를 활용해 태...,"[디스플레이, 경영, 강화, 목적, 한국전력, 녹색, 프리미엄, 제도, 활용, 태양...","[3, 10, 20]"
...,...,...,...,...,...
251508,499995,[21],아버지를 닮은 아버지 같은 해군 간부가 되겠다우석대 진천캠퍼스 군사안보학과 김해나 ...,"[아버지, 아버지, 해군, 간부, 석대, 진천, 캠퍼스, 군사, 보학, 김해, 학생...","[3, 2, 20]"
251509,499996,[9],평택시 기록관은 시민의 생활상과 추억이 담긴 민간기록물을 오는 10월 29일까지 수...,"[평택시, 기록관, 시민, 생활상, 추억, 민간, 기록물, 월, 일, 수집, 일, ...","[9, 14, 0]"
251510,499997,[9],국립낙동강생물자원관 연구진경북 상주시에 있는 환경부 산하 국립낙동강생물자원관이 최근...,"[국립, 낙동강, 생물, 자원, 관, 연구, 진경, 북, 상주시, 환경부, 산하, ...","[19, 2, 25]"
251511,499998,[29],총 상금 1200만 원 규모 8개 과채 소재 사진과 동영상 로 응모전북농협전북농협은...,"[상금, 만, 원, 규모, 개, 과채, 소재, 사진, 동영상, 응모, 전북, 농협,...","[14, 24, 12]"


In [55]:
data.to_csv('/workspace/data/git_category.csv')

In [13]:
import ast
import json

def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [58]:
temp_dict = [
    {"content": row['content'].strip(), "category": row['category']}
    for _, row in data.iterrows()
]

temp_dict[0]

{'content': '1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품의 부패를 방지하기 위해 산소 차단 기능이 적용된 포장재는 이전까지 일본산 소재인 에틸렌비닐알코올로만 만들어졌다 에버켐텍은 천연 단백질을 이용한 신소재로 일본의 식품포장재 시장 독점 구조를 깬 셈이다 에버켐텍이 소재부품장비 강소기업으로 평가받는 이유다2 2001년 설립된 영창케미칼은 일본의 3대 수출규제 품목 중 하나였던 포토레지스트 소재 국산화를 선도했다 반도체 공정의 필수 소재인 포토레지스트는 빛으로 회로 모양을 찍어내는 작업을 할 때 웨이퍼 위에 균일하게 도포되는 액체를 말한다 영창케미칼의 포토레지스트는 경쟁력을 인정받아 현재 삼성전자 SK하이닉스 등 국내 반도체 회사에 공급된다일본 경제산업성이 2019년 7월 고순도 불화수소 플루오린폴리이미드 포토레지스트에 대한 수출규제를 시작하면서 촉발된 소부장 기술전쟁이 2년째를 맞는다 정부는 소부장 기업들이 혁신과 정부 지원정책이 맞물려 기술 자립도가 높아졌다고 평가한다 100대 핵심 품목 대일 의존도 급감중소벤처기업부와 산업통상자원부에 따르면 최근 2년 동안 소부장 관련 100대 핵심 품목에 대한 대일 의존도는 줄고 소부장 기업의 매출은 20 넘게 증가했다 중기부가 선정한 소부장 강소기업100에 포함돼 있는 에버켐텍과 영창케미칼 등의 성과를 보면 일본의 수출규제가 소부장 기업들이 성장하는 계기가 된 셈이다소부장 기술독립의 성과는 양과 질적인 면에서 함께 나타나고 있다 시가총액이 1조 원 이 넘는 소부장 관련 중소중견기업 수는 2019년 13개에서 올해 31개로 늘었다소부장 생태계 내에서 대기업과 중소기업 간 협업 사례도 늘었다 중기부에 따르면 대기업은 생산 라인을 개방해 중소기업이 신규 기술을 검증받을 수 있도록 했다 그러면서 대기업은 중소기업이 개발한 소재를 공급받고 있다 이에 따라 지난해 1분기 소부장 상장기업의 총매출액은 2019년 1분기 대비 201 증가했다 이는 상장기업 전체 평균 매출액 증가율보다 

In [59]:
# df.to_json('/home/shpark/news_scraper/model/data/git_category.csv')
!rm -rf /workspace/data/git_category.json
dump_jsonl(temp_dict, '/workspace/data/git_category.json')

Wrote 251513 records to /workspace/data/git_category.json


## **len 1000**

In [5]:
data = pd.read_csv('/workspace/data/git_category.csv')

In [6]:
data = data[:1000]
data.count()

Unnamed: 0       1000
Unnamed: 0.1     1000
category_old     1000
content          1000
content_token    1000
category         1000
dtype: int64

In [7]:
data['category']

0         [2, 3, 9]
1       [3, 21, 22]
2       [5, 23, 13]
3       [23, 5, 29]
4       [3, 10, 20]
           ...     
995    [25, 18, 27]
996    [19, 25, 29]
997      [23, 5, 8]
998     [23, 10, 5]
999    [25, 19, 29]
Name: category, Length: 1000, dtype: object

In [8]:
data.to_csv('/workspace/data/git_category_1000.csv')

In [14]:
temp_dict = [
    {"content": row['content'].strip(), "category": ast.literal_eval(row['category'])}
    for _, row in data.iterrows()
]

temp_dict[0]

{'content': '1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품의 부패를 방지하기 위해 산소 차단 기능이 적용된 포장재는 이전까지 일본산 소재인 에틸렌비닐알코올로만 만들어졌다 에버켐텍은 천연 단백질을 이용한 신소재로 일본의 식품포장재 시장 독점 구조를 깬 셈이다 에버켐텍이 소재부품장비 강소기업으로 평가받는 이유다2 2001년 설립된 영창케미칼은 일본의 3대 수출규제 품목 중 하나였던 포토레지스트 소재 국산화를 선도했다 반도체 공정의 필수 소재인 포토레지스트는 빛으로 회로 모양을 찍어내는 작업을 할 때 웨이퍼 위에 균일하게 도포되는 액체를 말한다 영창케미칼의 포토레지스트는 경쟁력을 인정받아 현재 삼성전자 SK하이닉스 등 국내 반도체 회사에 공급된다일본 경제산업성이 2019년 7월 고순도 불화수소 플루오린폴리이미드 포토레지스트에 대한 수출규제를 시작하면서 촉발된 소부장 기술전쟁이 2년째를 맞는다 정부는 소부장 기업들이 혁신과 정부 지원정책이 맞물려 기술 자립도가 높아졌다고 평가한다 100대 핵심 품목 대일 의존도 급감중소벤처기업부와 산업통상자원부에 따르면 최근 2년 동안 소부장 관련 100대 핵심 품목에 대한 대일 의존도는 줄고 소부장 기업의 매출은 20 넘게 증가했다 중기부가 선정한 소부장 강소기업100에 포함돼 있는 에버켐텍과 영창케미칼 등의 성과를 보면 일본의 수출규제가 소부장 기업들이 성장하는 계기가 된 셈이다소부장 기술독립의 성과는 양과 질적인 면에서 함께 나타나고 있다 시가총액이 1조 원 이 넘는 소부장 관련 중소중견기업 수는 2019년 13개에서 올해 31개로 늘었다소부장 생태계 내에서 대기업과 중소기업 간 협업 사례도 늘었다 중기부에 따르면 대기업은 생산 라인을 개방해 중소기업이 신규 기술을 검증받을 수 있도록 했다 그러면서 대기업은 중소기업이 개발한 소재를 공급받고 있다 이에 따라 지난해 1분기 소부장 상장기업의 총매출액은 2019년 1분기 대비 201 증가했다 이는 상장기업 전체 평균 매출액 증가율보다 

In [15]:
dump_jsonl(temp_dict, '/workspace/data/git_category_1000.json')

Wrote 1000 records to /workspace/data/git_category_1000.json
